In [18]:
from raphtory import Graph
from raphtory import algorithms
from raphtory import graph_loader
import time
import sys
import pandas as pd
import altair as alt
from datetime import timezone, datetime

# Warning this loader will automatically download the full 11GB stablecoin dataset from SNAP if it not present at the path provided
# If you prefer to do this external to the notebook you can pre-download it from https://snap.stanford.edu/data/ERC20-stablecoins.zip
# Once downloaded please provide the path to the ERC20-stablecoins folder below

# The loader has an optional parameter 'subset' which if set to True will only ingest the first file "token_transfers.csv"
# This is suggested if running initial tests as it is only a 10th of the data
g = graph_loader.stable_coin_graph("/tmp/ERC20-stablecoins", subset=True)

Loaded graph with 1 shards from encoded data files /Users/bensteer/Desktop/ERC20-stablecoins/graphdb.bincode with 1523333 vertices, 2814155 edges which took 6 seconds


In [2]:
all_layers = g.get_unique_layers()
all_layers

['Dai', 'LUNC', 'USTC', 'USD', 'USDP', 'USDT']

In [3]:
for l in all_layers:
    print("Contract address {} has {} unique transacting pairs".format(l,g.layer(l).num_edges()))

Contract address Dai has 123876 unique transacting pairs
Contract address LUNC has 68811 unique transacting pairs
Contract address USTC has 41771 unique transacting pairs
Contract address USD has 1026779 unique transacting pairs
Contract address USDP has 4415 unique transacting pairs
Contract address USDT has 1605617 unique transacting pairs


In [4]:
pg = algorithms.pagerank(g, 20)

Done running iterations: 1


In [5]:
sorted_pg = dict(sorted(pg.items(), key=lambda x: x[1], reverse=True))
list(sorted_pg.items())[:10]

[('0x28c6c06298d514db089934071355e5743bf21d60', 0.07171340390501024),
 ('0x0d0707963952f2fba59dd06f2b425ace40b492fe', 0.02896946268324041),
 ('0x503828976d22510aad0201ac7ec88293211d23da', 0.019826180105657478),
 ('0x71660c4005ba85c37ccec55d0c4493e66fe775d3', 0.01920300164775567),
 ('0x3cd751e6b0078be393132286c442345e5dc49699', 0.018267600938864866),
 ('0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511', 0.018201210448803514),
 ('0x2faf487a4414fe77e2327f0bf4ae2a264a776ad2', 0.014134622881672275),
 ('0x88bd4d3e2997371bceefe8d9386c6b5b4de60346', 0.012854653398513542),
 ('0xec30d02f10353f8efc9601371f56e808751f396f', 0.01285194221467036),
 ('0x74de5d4fcbf63e00296fd95d33236b9794016631', 0.012491201438935555)]

In [6]:
for l in all_layers:
    print("Contract address {} has 0x28c6c06298d514db089934071355e5743bf21d60? {}".format(l, g.layer(l).has_vertex("0x28c6c06298d514db089934071355e5743bf21d60")))

Contract address Dai has 0x28c6c06298d514db089934071355e5743bf21d60? True
Contract address LUNC has 0x28c6c06298d514db089934071355e5743bf21d60? True
Contract address USTC has 0x28c6c06298d514db089934071355e5743bf21d60? True
Contract address USD has 0x28c6c06298d514db089934071355e5743bf21d60? True
Contract address USDP has 0x28c6c06298d514db089934071355e5743bf21d60? True
Contract address USDT has 0x28c6c06298d514db089934071355e5743bf21d60? True


In [7]:
for l in all_layers:
    print("Contract address {} has 0x74de5d4fcbf63e00296fd95d33236b9794016631? {}".format(l, g.layer(l).has_vertex("0x74de5d4fcbf63e00296fd95d33236b9794016631")))

Contract address Dai has 0x74de5d4fcbf63e00296fd95d33236b9794016631? True
Contract address LUNC has 0x74de5d4fcbf63e00296fd95d33236b9794016631? True
Contract address USTC has 0x74de5d4fcbf63e00296fd95d33236b9794016631? True
Contract address USD has 0x74de5d4fcbf63e00296fd95d33236b9794016631? True
Contract address USDP has 0x74de5d4fcbf63e00296fd95d33236b9794016631? True
Contract address USDT has 0x74de5d4fcbf63e00296fd95d33236b9794016631? True


In [8]:
for l in all_layers:
    pg = algorithms.pagerank(g.layer(l), 20)
    sorted_pg = dict(sorted(pg.items(), key=lambda x: x[1], reverse=True))
    print("Important people from contract address {}: {}".format(l, list(sorted_pg.items())[:10]))
    print()

Done running iterations: 1
Important people from contract address Dai: [('0x503828976d22510aad0201ac7ec88293211d23da', 0.0021117792682171037), ('0x28c6c06298d514db089934071355e5743bf21d60', 0.002109268326748625), ('0x74de5d4fcbf63e00296fd95d33236b9794016631', 0.001314686215381458), ('0x60594a405d53811d3bc4766596efd80fd545a270', 0.0009343411267338441), ('0x5777d92f208679db4b9778590fa3cab3ac9e2168', 0.0007376593290049379), ('0xa12431d0b9db640034b0cdfceef9cce161e62be4', 0.0005977745605573248), ('0x0000000000000000000000000000000000000000', 0.0005557580661770019), ('0x2faf487a4414fe77e2327f0bf4ae2a264a776ad2', 0.0004938959820280982), ('0xa478c2975ab1ea89e8196811f51a7b7ade33eb11', 0.00046649946263294117), ('0x66c57bf505a85a74609d2c83e94aabb26d691e1f', 0.0004587428014694163)]

Done running iterations: 1
Important people from contract address LUNC: [('0x77696bb39917c91a0c3908d577d5e322095425ca', 0.0021130216887443654), ('0x0000000000000000000000000000000000000000', 0.001041933171094226), ('0x

In [9]:
luna = g.layer("LUNC")

In [10]:
pg = algorithms.pagerank(luna, 20)

Done running iterations: 1


In [11]:
pg["0x0000000000000000000000000000000000000000"]

0.001041933171094226

In [12]:
for l in all_layers:
    for w in g.layer(l).rolling(step='1 day', window='1 day'):
        print("dt_time {}, Date {}, Contract address {} has {} transactions".format(datetime.fromtimestamp(w.start() / 1000), w.start_date_time(), l,w.layer(l).num_edges()))

dt_time 2022-04-28 01:30:15, Date 2022-04-28 00:30:15, Contract address Dai has 4691 transactions
dt_time 2022-04-29 01:30:15, Date 2022-04-29 00:30:15, Contract address Dai has 5237 transactions
dt_time 2022-04-30 01:30:15, Date 2022-04-30 00:30:15, Contract address Dai has 4828 transactions
dt_time 2022-05-01 01:30:15, Date 2022-05-01 00:30:15, Contract address Dai has 4266 transactions
dt_time 2022-05-02 01:30:15, Date 2022-05-02 00:30:15, Contract address Dai has 4504 transactions
dt_time 2022-05-03 01:30:15, Date 2022-05-03 00:30:15, Contract address Dai has 4354 transactions
dt_time 2022-05-04 01:30:15, Date 2022-05-04 00:30:15, Contract address Dai has 5202 transactions
dt_time 2022-05-05 01:30:15, Date 2022-05-05 00:30:15, Contract address Dai has 5336 transactions
dt_time 2022-05-06 01:30:15, Date 2022-05-06 00:30:15, Contract address Dai has 5475 transactions
dt_time 2022-05-07 01:30:15, Date 2022-05-07 00:30:15, Contract address Dai has 5263 transactions
dt_time 2022-05-08 0

In [13]:
events = [['2022-05-12', 'LUNA Crash']]
events = pd.DataFrame(events, columns=['Time', 'event'])
event_rules = alt.Chart(events).mark_rule(color='gray').encode(x='Time:T')
labels = alt.Chart(events).mark_text(dx=3, dy=-120, color='black', align='left').encode(
    x='Time:T',
    text='event'
)
event_chart = alt.layer(event_rules, labels)

In [14]:
result = pd.DataFrame()

all_layers = g.get_unique_layers()
avoid_layers = ['USD', 'USDT']
layers = set(all_layers) - set(avoid_layers)

rolling = g.rolling(step='1 day', window='1 day')
for l in layers:
    result[l] = [w.layer(l).num_edges() for w in rolling]

result['Time'] = [w.end_date_time() for w in rolling]

source = pd.melt(result, id_vars=['Time'], value_vars=layers, var_name='Layer', value_name='Tnx Count')
selection = alt.selection_multi(fields=['layer'], bind='legend')
chart = alt.Chart(source).mark_line().encode(
    x="Time:T",
    y=f"{'Tnx Count'}:Q",
    color="Layer",
    tooltip=['Layer', 'Tnx Count'],
    opacity=alt.condition(selection, alt.value(1.0), alt.value(0.2))
).properties(
    width=800,
).add_selection(
    selection
)

alt.layer(chart, event_chart)

alt.LayerChart(...)

In [15]:
result = pd.DataFrame()
all_layers = g.get_unique_layers()
avoid_layers = ['USD', 'USDT']
layers = set(all_layers) - set(avoid_layers)

rolling = g.rolling(step='1 day', window='1 week')
for l in layers:
    result[l] = [w.layer(l).num_edges() for w in rolling]

result['Time'] = [w.end_date_time() for w in rolling]

source = pd.melt(result, id_vars=['Time'], value_vars=layers, var_name='Layer', value_name='Tnx Count')
selection = alt.selection_multi(fields=['Layer'], bind='legend')
chart = alt.Chart(source).mark_line().encode(
    x="Time:T",
    y=f"{'Tnx Count'}:Q",
    color="Layer",
    tooltip=['Layer', 'Tnx Count'],
    opacity=alt.condition(selection, alt.value(1.0), alt.value(0.2))
).properties(
    width=800,
).add_selection(
    selection
)

alt.layer(chart, event_chart)

alt.LayerChart(...)

In [16]:
result = pd.DataFrame()

rolling = g.rolling(step='1 day', window='1 week')

result['Tnx Count'] = [w.layer('LUNC').num_edges() for w in rolling]
result['Time'] = [w.layer('LUNC').end_date_time() for w in rolling]

chart = alt.Chart(result).mark_line().encode(
    x="Time:T",
    y="Tnx Count:Q",
).properties(
    width=800,
).add_selection(
    selection
)

alt.layer(chart, event_chart)

alt.LayerChart(...)

In [17]:
result = pd.DataFrame()

rolling = g.rolling(step='1 day', window='1 week')

result['Page Rank'] = [algorithms.pagerank(w.layer('LUNC'), 20)["0xd30b438df65f4f788563b2b3611bd6059bff4ad9"] for w in rolling]
result['Time'] = [w.layer('LUNC').end_date_time() for w in rolling]

chart = alt.Chart(result).mark_line().encode(
    x="Time:T",
    y="Page Rank:Q",
).properties(
    width=800,
)

alt.layer(chart, event_chart)

Done running iterations: 1
Done running iterations: 1
Done running iterations: 1
Done running iterations: 1
Done running iterations: 1
Done running iterations: 1
Done running iterations: 1


thread '<unnamed>' panicked at 'index out of bounds: the len is 434423 but the index is 434423', /Users/bensteer/Documents/Raphtory/raphtory/src/db/task/task_runner.rs:214:13
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: index out of bounds: the len is 434423 but the index is 434423

In [ ]:
result = pd.DataFrame()

layers = g.get_unique_layers()
rolling = g.rolling(step='1 day', window='1 week')

for l in layers:
    result[l] = [algorithms.pagerank(w.layer(l), 20)["0x503828976d22510aad0201ac7ec88293211d23da"] for w in rolling]

result['Time'] = [w.end_date_time() for w in rolling]

source = pd.melt(result, id_vars=['Time'], value_vars=layers, var_name='Layer', value_name='Page Rank')
selection = alt.selection_multi(fields=['Layer'], bind='legend')

chart = alt.Chart(source).mark_line().encode(
    x="Time:T",
    y=f"{'Page Rank'}:Q",
    color="Layer",
    tooltip=['Layer', 'Page Rank'],
    opacity=alt.condition(selection, alt.value(1.0), alt.value(0.2))
).properties(
    width=800,
).add_selection(
    selection
)

alt.layer(chart, event_chart)

/Users/shivamkapoor/opt/miniconda3/envs/raphtory/lib/python3.10/site-packages/altair/utils/core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)